In [ ]:
'''Welcome to my submission for the first assignment in the Udacity 
Data Scientist Nanodegree: Write a Data Science Blog Post.

In this notebook I'll try to synthesize everything I've learned from my courses so far, 
using a dataset that I acquired myself, and even did most of the gathering for:
The data gathered by monitoring the growth and health of over 30,000 plants
seeded by an ecological restoration effort near Bogotá, Colombia across 5 geographically
distinct locations. For contractual reasons, I'll have to anonymize the exact
names and locations of the project, but rest assured, this is all real, 
on-the-ground data. Only a lunatic would make all of this up.

Please run cells sequentially.

'''

In [ ]:
'''Step 1: Business Understanding.
The fundamental goal of Ecological Restoration is to sow the seeds (figuratively and sometimes literally)
that take a geographic area from some sort of disturbance to a new,
balanced and functioning ecosystem.

In this case the ecological disturbance was caused by logging and extensive agriculture or industry,
the target for restoration is to replace these fields with something equivalent to 
the local ecosystem: Either Andean forest, or Páramo, depending on the local altitude.

To do this, we have chosen to assist the natural regeneration of native species, 
by sowing various species found in the adjacent forest and páramo in a few designs of
Nuclei: smaller grouped lots of varied species. This differs from a standard row planting
because it allows plants to take advantage of each other for cover, and is less expensive
for the same amount of area. It also mimics how plants tend to naturally colonize landscapes.
(See figure 1 for an example)

So now that we have everything planted, we must ask the following questions: 
    Mechanically - Is the process working? Are the plants growing properly? Are they surviving? 
    Are they healthy? Why or why not?
    
    Ecologically - Do we have a good representation of species? Are they outcompeting local species? 
    Are they properly situated? Why or why not?
    
    Logistically - What variables can we control to improve our outcomes? How can we do that?
    
Note: The Udacity project specifies 3-5 questions. Some of these are fairly simple to answer so
I'm choosing to take those three categories as compound questions to answer.
'''
#To do: Include an example picture of module layout as figure 1

In [ ]:
'''Step 2: Data Understanding.
The dataset covers 5 geographic areas, some of which are split based on various
ecological or organizational criteria, for a total of 9 excel files. Additionally,
we have separate tables containing the geographic coordinates of each Nucleus,
and a much dirtier dataset containing information about the origins of the plant material
from the nursery it came from.

In theory, these files should all have the same structure, column headings,
formatting, and have already been cleaned. A quick look at the files would show this is not the case.

'''

In [1]:
#Import all the necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
#Open and read all of the excel files. I have renamed these from the business'
#Actual filenames for ease of use and anonymity.

#First get the filenames from the directory
from os import walk
filenames = set()
for root, folders, files in walk('Original Data'):#Might not need the three levels
    for name in files:
        filenames.add(name)
        
filenames


{'Banqueo_Disturbed.xlsx',
 'Banqueo_Forest.xlsx',
 'Bojaca_Veda230.xlsx',
 'Bojaca_Veda500-675.xlsx',
 'Bojaca_Veda500-766.xlsx',
 'Coordinates.xlsx',
 'Esmeralda.xlsx',
 'Peñas.xlsx',
 'Reserve.xlsx',
 'Santa Rosa.xlsx'}

In [3]:
#Next actually get the data
#Create a dictionary of the lists.
df_dict = {}
#Assign data to dictionary
for name in filenames:
    text = name.replace('.xlsx', '')
    directory = f"Original Data\{name}"
    df_dict[text] = pd.read_excel(directory) #Unexpected benefit of using excel: easy unicode reading of chars like 'ñ'

In [4]:
#Some basic parameters of the data
for df in df_dict:
    print (df, df_dict[df].shape)
    
df_dict['Reserve'].head()

Banqueo_Forest (2000, 27)
Bojaca_Veda500-675 (2200, 26)
Santa Rosa (10980, 31)
Reserve (16506, 32)
Coordinates (54, 7)
Bojaca_Veda230 (2417, 24)
Peñas (6588, 28)
Bojaca_Veda500-766 (50, 25)
Esmeralda (11943, 28)
Banqueo_Disturbed (3505, 27)


,Fecha,ID,Even_mon,Especie,No_Mod,No_Ind,Alt1_Ind,Alt2_Ind,Alt3_Ind,Alt_Ind Promedio,...,Herbivoria,Patogenos,Competencia,Daño mecanico,Anegamiento,Planta no encontrada,Daño por Curi,Rebrotando,Observaciones_Sup,Period planted
0,2019-12-01 00:00:00,1,T1,Ceroxylon alpinum,1,1,39,0.0,0.0,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-01 00:00:00,2,T1,Ceroxylon alpinum,1,2,52,0.0,0.0,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T1
2,2019-12-01 00:00:00,3,T1,Ceroxylon alpinum,1,3,66,0.0,0.0,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T1
3,2019-12-01 00:00:00,4,T1,Ceroxylon alpinum,1,4,66,0.0,0.0,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T1
4,2019-12-01 00:00:00,5,T1,Ceroxylon alpinum,1,5,46,0.0,0.0,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
'''Clearly the data needs some cleaning. Looking at the sets also shows a lot of NaN values.
I know this from working with the data personally, but this is normal. The dataset is split into
categorical and context data (date, Even_mon, Especie, No_Mod), quantifiable data like
height (Alt_prom), stem diameter (DAB_prom), health (Est_Fit_Ind), and boolean data indicating survival
the affliction of the plant (Coloracion, Vigor Bajo). This data was taken in spanish so while
I will provide translation when necessary, please bear with me with everything else if the
column names don't make any sense.
''' 

In [ ]:
'''Step 3: Data Preparation
For the purposes of this analysis I will focus on a single one, which I will call
"Reserve", as it is the oldest and has the most data. 
Since each scenario is in a geographically distinct location, comparing between them
is difficult, and it would make sense to instead modularize the analysis and apply 
it repeatedly to each scenario.
'''

In [5]:
#Select the relevant datasets
Reserve = df_dict['Reserve']
Coords = df_dict['Coordinates']

In [6]:
#Looking at the dataset, we shouldn't have any object classes. Only datetime, string, int64, float64
print(Reserve.dtypes)

Fecha                    object
ID                        int64
Even_mon                 object
Especie                  object
No_Mod                    int64
No_Ind                    int64
Alt1_Ind                 object
Alt2_Ind                float64
Alt3_Ind                float64
Alt_Ind Promedio         object
DAB1_Ind                 object
DAB2_Ind                float64
DAB3_Ind                 object
DAB_prom                 object
Diam_1_Ind              float64
Diam_2_Ind              float64
Area_cop_ind            float64
Observaciones_Crec       object
Sup_Ind                  object
Est_Fit_Ind             float64
Coloracion               object
Vigor bajo               object
Herbivoria               object
Patogenos                object
Competencia              object
Daño mecanico            object
Anegamiento              object
Planta no encontrada     object
Daño por Curi            object
Rebrotando               object
Observaciones_Sup        object
Period p

In [7]:
#Convert data types
Reserve = Reserve.convert_dtypes() #This only does half of the columns
keys = ['Alt1_Ind','Alt_Ind Promedio','DAB1_Ind','DAB3_Ind','DAB_prom']
for item in keys:
    Reserve[item] = pd.to_numeric(Reserve[item], errors='coerce')
#To do: Good chance we can turn this into a function for every dataset

In [8]:
Reserve.dtypes #Much better, we can change Fecha after some examination.

Fecha                    object
ID                        Int64
Even_mon                 string
Especie                  string
No_Mod                    Int64
No_Ind                    Int64
Alt1_Ind                float64
Alt2_Ind                Float64
Alt3_Ind                Float64
Alt_Ind Promedio        float64
DAB1_Ind                float64
DAB2_Ind                Float64
DAB3_Ind                float64
DAB_prom                float64
Diam_1_Ind              Float64
Diam_2_Ind              Float64
Area_cop_ind            Float64
Observaciones_Crec       string
Sup_Ind                  string
Est_Fit_Ind               Int64
Coloracion               string
Vigor bajo               string
Herbivoria               string
Patogenos                string
Competencia              string
Daño mecanico            string
Anegamiento              string
Planta no encontrada     string
Daño por Curi            string
Rebrotando               string
Observaciones_Sup        string
Period p

In [9]:
Coords.head()

,Pastos 1,Pastos 2,Fajas 1,Fajas 2,Fajas 3,Bosque,Vegetacion
0,2.0,2.0,1.0,2.0,2.0,1,1.0
1,4.0,4.0,3.0,3.0,3.0,3,3.0
2,8.0,7.0,4.0,4.0,6.0,5,5.0
3,9.0,8.0,5.0,6.0,7.0,7,7.0
4,11.0,9.0,8.0,8.0,10.0,9,9.0


In [10]:
#It turns out that the coordinate data is in a different format than the rest of the data and is split into worksheets.
#we have to refactor that.
Coords = pd.read_excel('Original Data\Coordinates.xlsx', 'Reserve')
Coords.head()

,NOMBRE,ID_MUEST,COOR_ESTE,COOR_NORTE
0,Comp. Aprov Forestal - Arreglo 1,CAF-N1-1,-73.910047,4.793035
1,Comp. Aprov Forestal - Arreglo 1,CAF-N1-2,-73.910047,4.793082
2,Comp. Aprov Forestal - Arreglo 1,CAF-N1-3,-73.910006,4.793059
3,Comp. Aprov Forestal - Arreglo 1,CAF-N1-4,-73.910006,4.793011
4,Comp. Aprov Forestal - Arreglo 1,CAF-N1-5,-73.910047,4.792988


In [11]:
#Now each plant is given a unique identifier in the dataset
Reserve['ID'].unique().max() #It's 2751

#This is exactly 1/6 of the length of the set, we can conclude that each of the same plant was measured 6 times.
#This is corroborated by the unique monitoring events 'Even_mon' as well as unique dates 'Fecha'
#Though the date apparently has some formatting issues.
print(Reserve['Even_mon'].unique())
print(Reserve['Fecha'].unique())

<StringArray>
['T1', 'T2', 'T3', 'T4', 'T5', 'T6']
Length: 6, dtype: string
[datetime.datetime(2019, 12, 1, 0, 0) datetime.datetime(2020, 4, 1, 0, 0)
 datetime.datetime(2020, 7, 1, 0, 0) datetime.datetime(2020, 10, 1, 0, 0)
 '23/3/2021' datetime.datetime(2021, 7, 6, 0, 0)]


In [12]:
#So now we can format Fecha as datetime
Reserve['Fecha'] = pd.to_datetime(Reserve['Fecha'])
print(Reserve['Fecha'].unique())

['2019-12-01T00:00:00.000000000' '2020-04-01T00:00:00.000000000'
 '2020-07-01T00:00:00.000000000' '2020-10-01T00:00:00.000000000'
 '2021-03-23T00:00:00.000000000' '2021-07-06T00:00:00.000000000']


In [13]:
#The coordinate data only has resolution down to the sub-Nucleus level, not the individual.
#Luckily the dataset also assigns an identifier to the nucleus: No_Mod
print(Reserve['No_Mod'].unique())

#Similarly the coordinate data also includes this under 'ID_MUEST'
print(Coords['ID_MUEST'])

<IntegerArray>
[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
Length: 56, dtype: Int64
0      CAF-N1-1
1      CAF-N1-2
2      CAF-N1-3
3      CAF-N1-4
4      CAF-N1-5
         ...   
465      CAF-T9
466      CAF-T8
467      CAF-T6
468      CAF-T5
469      CAF-T7
Name: ID_MUEST, Length: 470, dtype: object


In [14]:
#However, Looking at the coordinate data shows that it has a lot of other data that is unrelated.
print(Coords['NOMBRE'].unique())

['Comp. Aprov Forestal - Arreglo 1' 'Comp. Aprov Forestal - Arreglo 2'
 'Comp. Aprov Forestal - Arreglo 3' 'Comp. Aprov Forestal - Arreglo 4'
 'Comp. Aprov Forestal - Arreglo 5' 'Comp. Aprov Forestal - Arreglo 6'
 'Comp. Aprov Forestal - Arreglo 7' 'Comp. Aprov Forestal - Arreglo 8'
 'Comp. Aprov Forestal - Arreglo 9' 'Comp. Aprov Forestal - Arreglo 10'
 'Comp. Aprov Forestal - Arreglo 11' 'Comp. Aprov Forestal - Arreglo 12'
 'Comp. Aprov Forestal - Arreglo 13' 'Comp. Aprov Forestal - Arreglo 14'
 'Comp. Aprov Forestal - Arreglo 15' 'Comp. Aprov Forestal - Arreglo 16'
 'Comp. Aprov Forestal - Arreglo 17' 'Comp. Aprov Forestal - Arreglo 18'
 'Comp. Aprov Forestal - Arreglo 19' 'Comp. Aprov Forestal - Arreglo 20'
 'Comp. Aprov Forestal - Arreglo 21' 'Comp. Aprov Forestal - Arreglo 22'
 'Comp. Aprov Forestal - Arreglo 23' 'Comp. Aprov Forestal - Arreglo 24'
 'Comp. Aprov Forestal - Arreglo 25' 'Comp. Aprov Forestal - Arreglo 26'
 'Comp. Aprov Forestal - Arreglo 27' 'Comp. Aprov Forestal -

In [15]:
#This refers to other scientific study data that is relevant to the project, but not to this analysis.
#We only want the data that contains the word "Arreglo" in the name, spanish for "Arrangement"
select_coords = Coords[Coords['NOMBRE'].str.contains('Arreglo')]

#We now have 7 coordinates for each Nucleus, so we need to average
#First by creating a new column attaching the nucleus number to each type
#We can get that directly from the 'ID_MUEST' column by applying some slicing:
nucs = select_coords['ID_MUEST'].str[5:-2]

select_coords['No_Mod'] = nucs #This way it has the same column name as the Reserve dataset
#Pandas drops a warning here but since we're not modifying the data anywhere in between we can ignore it.
select_coords.head()
#To do: We can probably save memory by doing this in a function.

<ipython-input-15-01ea9d3cd29d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_coords['No_Mod'] = nucs #This way it has the same column name as the Reserve dataset


,NOMBRE,ID_MUEST,COOR_ESTE,COOR_NORTE,No_Mod
0,Comp. Aprov Forestal - Arreglo 1,CAF-N1-1,-73.910047,4.793035,1
1,Comp. Aprov Forestal - Arreglo 1,CAF-N1-2,-73.910047,4.793082,1
2,Comp. Aprov Forestal - Arreglo 1,CAF-N1-3,-73.910006,4.793059,1
3,Comp. Aprov Forestal - Arreglo 1,CAF-N1-4,-73.910006,4.793011,1
4,Comp. Aprov Forestal - Arreglo 1,CAF-N1-5,-73.910047,4.792988,1


In [16]:
#Now we can average the coordinates
mask = select_coords.groupby(['No_Mod']).mean()
#Without these next 2 lines joining the tables throws a type error
mask.reset_index(level=0, inplace=True) 
mask['No_Mod'] = mask['No_Mod'].astype('int64')
mask
#To do: We can probably save memory by doing this in a function.

,No_Mod,COOR_ESTE,COOR_NORTE
0,1,-73.910047,4.793035
1,10,-73.909950,4.792362
2,11,-73.909673,4.792434
3,12,-73.909547,4.792267
4,13,-73.909824,4.792175
5,14,-73.909731,4.791990
6,15,-73.909486,4.791552
7,16,-73.909415,4.791411
8,17,-73.909428,4.791240
9,18,-73.909707,4.790889


In [17]:
#Now we can finally merge the data
Reserve = Reserve.merge(mask, how='outer', on='No_Mod')
#To do: We can probably save memory by doing this in a function.

#Verify the merge
Reserve[['ID', 'COOR_ESTE', 'COOR_NORTE']].head()


,ID,COOR_ESTE,COOR_NORTE
0,1,-73.910047,4.793035
1,2,-73.910047,4.793035
2,3,-73.910047,4.793035
3,4,-73.910047,4.793035
4,5,-73.910047,4.793035


In [18]:
#Now that all of our data is in the same table, let's check for errors
#We can only realistically check for spelling errors and categorical errors, not data errors

print(Reserve['Especie'].unique())

#The correct format for a scientific name is 'Genus species', with 'sp.' instead of the species when unknown.

Reserve['Especie'] = Reserve['Especie'].str.capitalize()

#'Sieinmannia sp' is a spelling mistake. It refers to 'Weinmannia sp.' Similarly not every sp has a period.

Reserve['Especie'].replace(to_replace='Sieinmannia sp', value='Weinmannia sp.', inplace=True)
Reserve['Especie'].replace(to_replace='Weinmannia sp', value='Weinmannia sp.', inplace=True)


<StringArray>
[        'Ceroxylon alpinum',       'Tibouchina lepidota',
         'Morella pubescens',        'Lupinus bogotensis',
     'Escallonia paniculata',      'Mycianthes leucoxyla',
      'Ageratina boyacensis',         'Viburnum tinoides',
        'Juglans neotropica',     'Verbesina crassiramea',
       'Hesperomeles nitida',         'Vallea stipularis',
        'Myrsine guianensis',         'Clusia multiflora',
       'Baccharis latifolia',          'Hesperomeles sp.',
           'Cedrela montana',           'Piper nubigenum',
         'Myrsine dependens',        'Escallonia pendula',
         'Oreopanax incisus',           'Cestrum mutisii',
                         '*',   'Hesperomeles goudotiana',
 'Retrophyllum rospigliosii',       'Bucquetia glutinosa',
     'Ageratina asclepiadea',      'ageratina boyacensis',
   'Ageratina glyptophlebia',        'lupinus bogotensis',
             'Weinmannia sp',            'SIeinmannia sp',
             'Ageratina sp.',             

In [20]:
#There should be no null values for species

print(Reserve[Reserve['Especie'].isnull() == True]) #One is a complete empty row. No idea where that came from. The other can be inferred

Reserve.drop(index = 16506, inplace=True) #Drop the empty row

Reserve[Reserve['ID'] == 2551] #Looking at the same ID from previous, we can glean that it was a Cedrela that died.

Reserve.iloc[15528, Reserve.columns.get_loc('Especie')] = 'Cedrela montana' #Fix empty
#To do: Evaluate if there's no better way to do this and the cell below. 

           Fecha    ID Even_mon Especie No_Mod  No_Ind  Alt1_Ind  Alt2_Ind  \
15528 2021-03-23  2551       T5    <NA>     53    2551       NaN      <NA>   
16506        NaT  <NA>     <NA>    <NA>     57    <NA>       NaN      <NA>   

       Alt3_Ind  Alt_Ind Promedio  ...  Competencia  Daño mecanico  \
15528      <NA>               0.0  ...         <NA>           <NA>   
16506      <NA>               NaN  ...         <NA>           <NA>   

       Anegamiento  Planta no encontrada  Daño por Curi  Rebrotando  \
15528         <NA>                  <NA>           <NA>        <NA>   
16506         <NA>                  <NA>           <NA>        <NA>   

       Observaciones_Sup Period planted  COOR_ESTE  COOR_NORTE  
15528               <NA>             T5 -73.910672    4.792448  
16506               <NA>           <NA> -73.910337    4.792404  

[2 rows x 34 columns]


In [21]:
#Do the same for species values of '*'
print(Reserve[Reserve['Especie'] == '*'])

Reserve[Reserve['ID'] == 160]
Reserve.iloc[1090, Reserve.columns.get_loc('Especie')] = 'Tibouchina lepidota'

Reserve[Reserve['ID'] == 269]
Reserve.iloc[1689, Reserve.columns.get_loc('Especie')] = 'Lupinus bogotensis'

          Fecha   ID Even_mon Especie No_Mod  No_Ind  Alt1_Ind  Alt2_Ind  \
1090 2021-03-23  160       T5       *      4     160       NaN      <NA>   
1689 2021-03-23  269       T5       *      6     269       NaN      <NA>   

      Alt3_Ind  Alt_Ind Promedio  ...  Competencia  Daño mecanico  \
1090      <NA>               0.0  ...         <NA>           <NA>   
1689      <NA>               0.0  ...         <NA>           <NA>   

      Anegamiento  Planta no encontrada  Daño por Curi  Rebrotando  \
1090         <NA>                  <NA>           <NA>        <NA>   
1689         <NA>                  <NA>           <NA>        <NA>   

      Observaciones_Sup Period planted  COOR_ESTE  COOR_NORTE  
1090               <NA>             T5 -73.910237    4.792702  
1689               <NA>             T5 -73.909952    4.792859  

[2 rows x 34 columns]


In [32]:
'''There is one final snag in modelling this dataset: replacement.
As part of our contract, we were required to plant exactly 2751 individuals,
and to replace them in the event of their death. This means that there are several
rows that cannot be compared from one time of sampling to the next.
As an additional complication, records kept of which species were replanted and where
are not clear. Some is already available in the column "Period planted"
but this was added retroactively, with the more difficult ones to verify left blank.
We will therefore try to fix this here.'''
reserve_copy = Reserve
#First - External knowledge here, There were replants just before T3 and T5, so anything before T3 is original
Reserve.loc[Reserve["Even_mon"]=="T1", "Period planted"] = "T1"
Reserve.loc[Reserve["Even_mon"]=="T2", "Period planted"] = "T1"



Empty DataFrame
Columns: [Fecha, ID, Even_mon, Especie, No_Mod, No_Ind, Alt1_Ind, Alt2_Ind, Alt3_Ind, Alt_Ind Promedio, DAB1_Ind, DAB2_Ind, DAB3_Ind, DAB_prom, Diam_1_Ind, Diam_2_Ind, Area_cop_ind, Observaciones_Crec, Sup_Ind, Est_Fit_Ind, Coloracion, Vigor bajo, Herbivoria, Patogenos, Competencia, Daño mecanico, Anegamiento, Planta no encontrada, Daño por Curi, Rebrotando, Observaciones_Sup, Period planted, COOR_ESTE, COOR_NORTE]
Index: []

[0 rows x 34 columns]


In [47]:
#Next, we generate a dictionary of which plants died from one period to the next
dead = {}
for period in Reserve['Even_mon'].unique():
    dead[str(period)] = Reserve.loc[(Reserve['Even_mon'] == str(period)) & (Reserve['Sup_Ind'] == 'NO')]
    

In [55]:
for item in dead:
    print(f'{item}, {dead[item].shape}')
#The low mortality in T3 and T5 tells us that this was indeed probably because of replacement
#Mortality also appears to decrease significantly over time. This is good.
#We can also conclude that any plant that died before T3 was replaced during T3
#And that any that died in T4 were replaced during T5

T1, (37, 34)
T2, (1248, 34)
T3, (0, 34)
T4, (764, 34)
T5, (9, 34)
T6, (270, 34)


In [81]:
#We need to verify that the dead plants in T1 and T5 were measured again in T2 and T6 respecively
t1_deadlist = dead['T1']['ID']
t2_deadlist = dead['T2']['ID']
t5_deadlist = dead['T5']['ID']
t6_deadlist = dead['T5']['ID']

t1_t2_int = set(t1_deadlist).intersection(set(t2_deadlist))
t1_t2_diff = set(t1_deadlist).difference(set(t2_deadlist))
t5_t6_int = set(t5_deadlist).intersection(set(t6_deadlist))

print(t1_t2_int)
print(t1_t2_diff)
print(t5_t6_int)
set(t6_deadlist) == t5_t6_int #returns True
#T6 dead includes T5 dead. T2 dead only includes some of T1 dead.

{2341, 550, 553, 171, 2736, 577, 73, 1998, 2263, 1240, 217, 734, 1888, 1250, 1890, 1640, 2033, 1651, 2037}
{706, 611, 68, 583, 72, 584, 330, 586, 587, 366, 432, 1937, 464, 2673, 212, 149, 213, 341}
{160, 2689, 71, 74, 13, 269, 466, 2551, 89}


True

In [78]:
#Is there a chance that the T1 dead that are not in T2 were simply a sampling mistake?
#If the IDs of t1_t2_diff are similar to dead that exist in t2, we might conclude that there was a mistaken ID in the field

mask = set()
for item in t1_t2_diff:
    
    approx = set([item + 1, item + 2, item +3, item -1, item -2, item-3])
    mask.update(approx)
approximate_values = mask.intersection(set(t2_deadlist))

In [79]:
print(sorted(t1_t2_diff))
print(sorted(approximate_values))
#There isn't a good correlation of values in these sets that would indicate that this was a systematic error
#Therefore it's probably best that we just exclude the t1_t2_diff rows from relevant analysis.

[68, 72, 149, 212, 213, 330, 341, 366, 432, 464, 583, 584, 586, 587, 611, 703, 704, 705, 706, 707, 708, 709, 1937, 2673]
[71, 73, 146, 147, 151, 327, 329, 340, 344, 369, 435, 580, 581, 585, 588, 608, 709, 2675]


In [113]:
Reserve[(Reserve["Even_mon"]=="T3") & (Reserve['Sup_Ind'] == True)]

,Fecha,ID,Even_mon,Especie,No_Mod,No_Ind,Alt1_Ind,Alt2_Ind,Alt3_Ind,Alt_Ind Promedio,...,Competencia,Daño mecanico,Anegamiento,Planta no encontrada,Daño por Curi,Rebrotando,Observaciones_Sup,Period planted,COOR_ESTE,COOR_NORTE
98,2020-07-01,1,T3,Ceroxylon alpinum,1,1,26.0,0.0,0.0,26.000000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910047,4.793035
99,2020-07-01,2,T3,Ceroxylon alpinum,1,2,31.0,0.0,0.0,31.000000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910047,4.793035
100,2020-07-01,3,T3,Ceroxylon alpinum,1,3,13.0,22.0,0.0,17.500000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910047,4.793035
101,2020-07-01,4,T3,Ceroxylon alpinum,1,4,23.0,16.0,0.0,19.500000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910047,4.793035
102,2020-07-01,5,T3,Ceroxylon alpinum,1,5,33.0,0.0,0.0,33.000000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910047,4.793035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16354,2020-07-01,2740,T3,Ageratina glyptophlebia,56,2740,89.0,72.0,19.0,60.000000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910439,4.792561
16355,2020-07-01,2741,T3,Ageratina glyptophlebia,56,2741,66.0,0.0,0.0,66.000000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910439,4.792561
16356,2020-07-01,2742,T3,Ageratina glyptophlebia,56,2742,57.0,49.0,42.0,49.333333,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910439,4.792561
16357,2020-07-01,2743,T3,Ageratina glyptophlebia,56,2743,89.0,70.0,0.0,79.500000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,T3,-73.910439,4.792561


In [110]:
#For T3 we can conclude that anything dead in T2 was replanted, but leave T1 out.
deadmask = Reserve['Sup_Ind'].mask(Reserve['Sup_Ind'] == 'NO')
deadmask = deadmask.notnull()
Reserve['Sup_Ind'] = deadmask
Reserve.loc[(Reserve["Even_mon"]=="T3") & (Reserve['Sup_Ind'] == True), "Period planted"] = "T3"
#Continue this later. What we did was change values at the wrong time, so T3 suddenly became all planted at T3
#What we need to do is transpose the mask 1 generation, so what was dead previously and is now alive becomes T3
#But first redo all the changes because T3 has incorrect data now.

In [22]:
'''We finally have our clean(ish) dataset for now.
What remains is to write it to a file so we can save it separately.'''

writer = pd.ExcelWriter('Reserve.xlsx', engine='openpyxl')  #The directory is different so we don't overwrite
Reserve.to_excel(writer, index = False, engine='openpyxl')

In [30]:
Reserve.columns

Index(['Fecha', 'ID', 'Even_mon', 'Especie', 'No_Mod', 'No_Ind', 'Alt1_Ind',
       'Alt2_Ind', 'Alt3_Ind', 'Alt_Ind Promedio', 'DAB1_Ind', 'DAB2_Ind',
       'DAB3_Ind', 'DAB_prom', 'Diam_1_Ind', 'Diam_2_Ind', 'Area_cop_ind',
       'Observaciones_Crec', 'Sup_Ind', 'Est_Fit_Ind', 'Coloracion',
       'Vigor bajo', 'Herbivoria', 'Patogenos', 'Competencia', 'Daño mecanico',
       'Anegamiento', 'Planta no encontrada', 'Daño por Curi', 'Rebrotando',
       'Observaciones_Sup', 'Period planted', 'COOR_ESTE', 'COOR_NORTE'],
      dtype='object')

In [ ]:
'''Part 4: Data Modeling'''